In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import cv2
import pandas as pd

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.mylayer import mylayer
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

K.set_learning_phase(1) #set learning phase

# sys.setrecursionlimit(40000)

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
network ='vgg'

if network == 'vgg':
	from keras_frcnn import vgg as nn
elif network == 'resnet50':
	from keras_frcnn import resnet as nn
else:
	print('Not a valid model')
	raise ValueError


In [3]:
from keras_frcnn.RoiPoolingConv import RoiPoolingConv
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (400, 400, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(6, 4))
nb_classes = 575

#pdb.set_trace()
num_rois = 6
# compile times on theano tend to be very high, so we use smaller ROI pooling regions to workaround

if K.backend() == 'tensorflow':
    pooling_regions = 7
    input_shape = (num_rois,7,7,512)
elif K.backend() == 'theano':
    pooling_regions = 7
    input_shape = (num_rois,512,7,7)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)


out_roi_pool = mylayer(pooling_regions, 6)([shared_layers, roi_input])
print out_roi_pool

out = (Flatten(name='flatten'))(out_roi_pool)
out = (Dense(4096, activation='relu', name='fc1'))(out)
out = (Dropout(0.5))(out)
out = (Dense(2048, activation='relu', name='fc2'))(out)
out = (Dropout(0.5))(out)
print 'no of classes', nb_classes
out_class = (Dense(166, activation='sigmoid', kernel_initializer='zero'))(out)

model_classifier = Model([img_input, roi_input], out_class)


try:
	print('loading weights from {}'.format(C.base_net_weights))
	model_classifier.load_weights(C.base_net_weights, by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')

optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_classifier.compile(optimizer=optimizer_classifier, loss = 'binary_crossentropy')



Instructions for updating:
Colocations handled automatically by placer.
Tensor("mylayer_1/Reshape:0", shape=(1, 6, 7, 7, 512), dtype=float32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
no of classes 575
Could not load pretrained model weights. Weights can be found in the keras application folder 		https://github.com/fchollet/keras/tree/master/keras/applications


In [13]:
def name(x):
    name = str.split(x,'/')
    return '/Users/1024329/Downloads/DeepFashion/img_n/' + name[1] + '/' + name[2]
    
df3 = pd.read_pickle('upper_body_atts.pkl')
df3['img_name'] = df3['img_name'].apply(lambda x:name(x))

In [14]:

img = []
landmarks = []
attribs = []
for i in range(100):
    #print i
    a = cv2.imread(df3.loc[i,'img_name'])
    a = cv2.resize(a,(400,400))
    img.append(a)
    lnd = df3.loc[i,'landmarks']
    landmarks.append(lnd)
    atts = df3.loc[i,'atts']
    attribs.append(atts)

In [18]:
df3.head(2)

,Unnamed: 0,x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6,w,h,img_name,landmarks,atts
0,24870,29,45,76,21,68.0,13.0,11,11,63,67,124.0,121.0,92,137,/Users/1024329/Downloads/DeepFashion/img_n/Lon...,"[[96, 2, 60, 60], [165, 2, 60, 60], [300, 153,...","[99, 495, 571, 881, 940]"
1,106141,53,112,22,130,44.0,117.0,19,18,122,137,191.0,195.0,157,216,/Users/1024329/Downloads/DeepFashion/img_n/Hoo...,"[[105, 5, 60, 60], [255, 3, 60, 60], [26, 195,...","[470, 953]"


In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

X = np.array(img)
X2 = np.array(landmarks)
Y = mlb.fit_transform(attribs)
X2 = (X2/4).astype(int)
print 'shapes of input and output', X.shape, X2.shape, Y.shape


shapes of input and output (100, 400, 400, 3) (100, 6, 4) (100, 166)


In [ ]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)
model_classifier.fit(x = [X,X2],y= Y, batch_size=1, epochs=2,validation_split=0.2,callbacks = [checkpoint])